In [270]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [271]:
df = pd.read_csv('energydata_complete.csv')


In [272]:
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [273]:
# sample of T2 (Temperature of the living room) and T6 (Temperature outside the biulding) from the dataset
sample_t2_t6_df = df[['T2', 'T6']]
sample_t2_t6_df.head()


,T2,T6
0,19.2,7.026667
1,19.2,6.833333
2,19.2,6.560000
3,19.2,6.433333
4,19.2,6.366667


In [274]:
x = sample_t2_t6_df[['T2']]

In [275]:
y = sample_t2_t6_df[['T6']]

In [276]:
linear_model = LinearRegression()

In [277]:
linear_model.fit(x, y)

LinearRegression()

In [278]:
# R-Squared value of T2 (Temperature of the living room) and T6 (Temperature outside the biulding)
linear_model.score(x, y).round(2)

0.64

In [279]:
scaler = MinMaxScaler()

In [280]:
df.drop(columns=['date', 'lights'], inplace=True)

In [281]:
normalized_df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

In [282]:
target_variable = normalized_df['Appliances']

In [283]:
features_df = normalized_df.drop(columns=['Appliances'])

In [284]:
# Splitting the dataset into training and testing datasets
x_train, x_test, y_train, y_test = train_test_split(
    features_df, target_variable, test_size=0.3, random_state=42)

In [285]:
# Fitting the model into the training dataset
linear_model.fit(x_train, y_train)

LinearRegression()

In [286]:
# Evaluating model on the test set
predicted_values = linear_model.predict(x_test)

In [287]:
# Mean Absolute Error (MAE)
mean_absolute_error(y_test, predicted_values).round(2)

0.05

In [288]:
# Residual Sum of Squares
np.sum(np.square(predicted_values - y_test)).round(2)

45.34

In [289]:
# Root Mean Square (RMSE)
np.sqrt(mean_squared_error(y_test, predicted_values)).round(3)

0.088

In [290]:
# Coefficient of Determination
r2_score(y_test, predicted_values).round(2)

0.15

In [291]:
def get_weights_df(model, feat, col_name):
    # this function returns the weights of every feature
    weights = pd.Series(model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['Features', col_name]
    weights_df[col_name].round(3)
    return weights_df

In [292]:
get_weights_df(linear_model, x_train, 'Linear_Model_Weight')

,Features,Linear_Model_Weight
0,rv2,-2.219382e+11
1,RH_2,-4.567699e-01
2,T_out,-3.220071e-01
3,T2,-2.361966e-01
4,T9,-1.899221e-01
5,RH_8,-1.576066e-01
6,RH_out,-7.770744e-02
7,RH_7,-4.455795e-02
8,RH_9,-3.980120e-02
9,T5,-1.567787e-02


In [293]:
ridge_reg = Ridge(alpha=0.4)

In [294]:
ridge_reg.fit(x_train, y_train)

Ridge(alpha=0.4)

In [295]:
ridge_predicted_values = ridge_reg.predict(x_test)

In [296]:
# Mean Absolute Error (MAE)
# There is no change to the Root Mean Squared Error (RMSE)
mean_absolute_error(y_test, ridge_predicted_values).round(2)

0.05

In [297]:
# Training a lasso regression model
lasso_reg = Lasso(alpha=0.001)

In [298]:
lasso_reg.fit(x_train, y_train)

Lasso(alpha=0.001)

In [299]:
lasso_df = get_weights_df(lasso_reg, x_train, 'Lasso_Model_weight')

In [300]:
lasso_df.head()

,Features,Lasso_Model_weight
0,RH_out,-0.049557
1,RH_8,-0.000110
2,T1,0.000000
3,Tdewpoint,0.000000
4,Visibility,0.000000


In [301]:
# 4 of the features have non-zero weights
len(lasso_df[lasso_df['Lasso_Model_weight'] != 0])

4

In [302]:
lasso_predicted_values = lasso_reg.predict(x_test)

In [303]:
# lasso RMSE
np.sqrt(mean_squared_error(y_test, lasso_predicted_values)).round(3)

0.094